In [3]:
import os
import librosa
import soundfile as sf
import numpy as np
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from typing import List, Tuple


c:\Users\syzygy\anaconda3\envs\new\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.models.wav2vec2.processing_wav2vec2 because of the following error (look up to see its traceback):
operator torchvision::nms does not exist

In [2]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

NameError: name 'Wav2Vec2Processor' is not defined

In [4]:
def load_and_resample(path: str, target_sr=16000) -> np.ndarray:
    audio, sr = librosa.load(path, sr=None)
    if sr != target_sr:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)
    return audio

def segment_audio(audio: np.ndarray, sample_rate: int, win_sec: float = 0.20, hop_sec: float = 0.10) -> List[np.ndarray]:
    win_size = int(sample_rate * win_sec)
    hop_size = int(sample_rate * hop_sec)
    segments = []
    for start in range(0, len(audio) - win_size + 1, hop_size):
        segment = audio[start:start + win_size]
        segments.append(segment)
    return segments


NameError: name 'List' is not defined

In [5]:
def extract_wav2vec_features(segments: List[np.ndarray]) -> List[np.ndarray]:
    features = []
    for seg in segments:
        inputs = processor(seg, sampling_rate=16000, return_tensors="pt", padding=True)
        input_values = inputs.input_values.to(device)
        with torch.no_grad():
            hidden_states = model(input_values).last_hidden_state  # shape: (1, time, 768)
            mean_feat = hidden_states.mean(dim=1).squeeze(0).cpu().numpy()  # shape: (768,)
            features.append(mean_feat)
    return features


NameError: name 'List' is not defined

In [6]:
def process_audio_file(path: str, output_dir: str, clip_id_prefix: str):
    os.makedirs(output_dir, exist_ok=True)
    audio = load_and_resample(path)
    segments = segment_audio(audio, 16000, win_sec=2.0, hop_sec=1.0)
    features = extract_wav2vec_features(segments)

    for i, (seg, feat) in enumerate(zip(segments, features)):
        seg_path = os.path.join(output_dir, f"{clip_id_prefix}_{i:03d}.wav")
        feat_path = os.path.join(output_dir, f"{clip_id_prefix}_{i:03d}.npy")
        sf.write(seg_path, seg, 16000)      # 保存音频段
        np.save(feat_path, feat)            # 保存特征向量
    print(f"Saved {len(segments)} clips and features to {output_dir}")


In [ ]:
if __name__ == "__main__":
    input_audio = "data/raw_audio/trump_speech.wav"
    output_folder = "data/processed/trump_clips"
    process_audio_file(input_audio, output_folder, "trump")
